# The Big Five personality traits (OCEAN) 


-Openness to experience  
-Conscientiouness  
-Extroversion  
-Agreeableness  
-Neurocitism  


1-import data
2-clean data
3-group data(similarity in answers) 5 clusters (Elbow Method)

4-answer the test and kmeans says in which group(cluster) I am


https://www.kaggle.com/datasets/tunguz/big-five-personality-test

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import open
pd.options.display.max_columns = 150


In [ ]:
data = pd.read_csv('data-final.csv', sep='\t')

In [ ]:
data.drop(data.columns[50:], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
pd.options.display.float_format = "{:.2f}".format
data.describe()

In [ ]:
data["EXT1"].value_counts()

In [ ]:
data = data[(data > 0.00).all(axis=1)]

In [ ]:
data["EXT1"].value_counts()

In [ ]:
pip install yellowbrick

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,10))

complete data is too big, so separate a sample

In [ ]:
data_sample = data.sample(n=8000, random_state=1)

In [ ]:
visualizer.fit(data_sample)
visualizer.poof()

In [ ]:
kmeans = KMeans(n_clusters=5)
k_fit = kmeans.fit(data)

In [ ]:
prediction = k_fit.labels_
data['Clusters'] = prediction

In [ ]:
data.head()

Quantity of observation in each group?

In [ ]:
data["Clusters"].value_counts()

In [ ]:
data.groupby('Clusters').mean()

Mean of each group

In [ ]:
col_list = list(data)
ext = col_list[0:10] #ext = extroversion
neu = col_list[10:20] #neu = neurotic
agr = col_list[20:30] #agr = agreeableness
csn = col_list[30:40] #csn = conscientious
opn = col_list[40:50] #opn = open 

In [ ]:
data_plus = pd.DataFrame()
data_plus['extroversion'] = data[ext].sum(axis=1)/10
data_plus['neurotic'] = data[neu].sum(axis=1)/10
data_plus['agreeable'] = data[agr].sum(axis=1)/10
data_plus['conscientious'] = data[csn].sum(axis=1)/10
data_plus['open'] = data[opn].sum(axis=1)/10
data_plus['clusters'] = prediction


In [ ]:
data_plus.groupby('clusters').mean()

mean by group

In [ ]:
data_clusters = data_plus.groupby('clusters').mean()

In [ ]:
plt.figure(figsize=(21,3))
for i in range(0, 5):
    plt.subplot(1,5,i+1)
    plt.bar(data_clusters.columns, data_clusters.iloc[:, i], color='green', alpha=0.2)
    plt.plot(data_clusters.columns, data_clusters.iloc[:, i], color='red')
    plt.title('Group ' + str(i))
    plt.xticks(rotation=45)
    plt.ylim(0,4);
    

# Read questions

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
dict_questions = open("questions.txt").read().split("\n")


In [ ]:
dict_questions

In [ ]:
questions = []
for q in dict_questions:
  q = str(q)
  questions.append(q[q.find("\t"):].lstrip())

In [ ]:
questions

In [ ]:
inputs_questions = []
for q in questions:
  obj_input = gr.inputs.Slider(minimum=1,maximum=5,step=1,default=3,label=q)
  inputs_questions.append(obj_input)
    

In [ ]:
def predict(*outputs_questions):
    outputs_questions = np.array(outputs_questions).reshape(1, -1)
    return k_fit.predict(outputs_questions)

iface = gr.Interface(
                    fn = predict,
                    title = "Big Five Personality Trait",
                    description = "Personality test",
                    inputs = inputs_questions,
                    outputs= "text")
iface.launch(share=True)